## Deep Learning

In [46]:
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping

## Import Data

In [4]:
data = pd.read_csv("review_sentiment.csv")

In [5]:
data.head()

,Unnamed: 0,review_text,restaurant_label,food_score,service_score,ambiance_score,authentic_score,overall_score,sentiment
0,0,come entre nous periodically past year seem se...,0,0.462757,0.427331,0.331545,0.492026,0.428415,1
1,1,absolutely delicious menu lovely offer excepti...,0,0.448536,0.318372,0.282956,0.371091,0.355239,1
2,2,thank much choose entre nous creperie recently...,0,0.430254,0.381057,0.289790,0.406790,0.376973,1
3,3,lovely little french restaurant really authent...,0,0.416918,0.309187,0.258666,0.506409,0.372795,1
4,4,dear rebecca thank much dining entre nous crep...,0,0.414757,0.294917,0.352158,0.438276,0.375027,1


In [6]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40946 entries, 0 to 40945
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   review_text       40946 non-null  object 
 1   restaurant_label  40946 non-null  int64  
 2   food_score        40946 non-null  float64
 3   service_score     40946 non-null  float64
 4   ambiance_score    40946 non-null  float64
 5   authentic_score   40946 non-null  float64
 6   overall_score     40946 non-null  float64
 7   sentiment         40946 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 2.5+ MB


## Train and modelling

In [9]:
X = data['review_text']
y = data['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Long Short-Term Memory

In [10]:
# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=100, truncating="post")

# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_length=100))
model.add(LSTM(100))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, validation_split=0.2)

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, truncating="post")
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/5
717/717 [==============================] - 25s 34ms/step - loss: 0.4213 - accuracy: 0.8032 - val_loss: 0.3532 - val_accuracy: 0.8416
Epoch 2/5
717/717 [==============================] - 24s 34ms/step - loss: 0.3180 - accuracy: 0.8634 - val_loss: 0.3525 - val_accuracy: 0.8439
Epoch 3/5
717/717 [==============================] - 25s 34ms/step - loss: 0.2883 - accuracy: 0.8789 - val_loss: 0.3567 - val_accuracy: 0.8427
Epoch 4/5
717/717 [==============================] - 25s 35ms/step - loss: 0.2615 - accuracy: 0.8942 - val_loss: 0.3742 - val_accuracy: 0.8369
Epoch 5/5
384/384 [==============================] - 4s 10ms/step - loss: 0.3901 - accuracy: 0.8390
Test Accuracy: 0.8389775156974792


In [ ]:
## With dropout

In [12]:
# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=100, truncating="post")

# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_length=100))
model.add(LSTM(100, return_sequences=True))  # Return sequences if stacking LSTM layers
model.add(Dropout(0.2))  # Add dropout layer with 20% dropout rate
model.add(LSTM(100))
model.add(Dropout(0.2))  # Add dropout layer with 20% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, validation_split=0.2)

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, truncating="post")
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/5
717/717 [==============================] - 57s 77ms/step - loss: 0.4182 - accuracy: 0.8074 - val_loss: 0.3603 - val_accuracy: 0.8400
Epoch 2/5
717/717 [==============================] - 56s 78ms/step - loss: 0.3127 - accuracy: 0.8640 - val_loss: 0.3944 - val_accuracy: 0.8186
Epoch 3/5
717/717 [==============================] - 55s 77ms/step - loss: 0.2773 - accuracy: 0.8832 - val_loss: 0.3735 - val_accuracy: 0.8357
Epoch 4/5
717/717 [==============================] - 55s 77ms/step - loss: 0.2467 - accuracy: 0.8990 - val_loss: 0.3786 - val_accuracy: 0.8394
Epoch 5/5
384/384 [==============================] - 8s 20ms/step - loss: 0.4309 - accuracy: 0.8334
Test Accuracy: 0.8334418535232544


In [13]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_length=100))
model.add(LSTM(200, return_sequences=True))  # Return sequences if stacking LSTM layers
model.add(Dropout(0.3))  # Add dropout layer with 30% dropout rate
model.add(LSTM(200))
model.add(Dropout(0.2))  # Add dropout layer with 20% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, validation_split=0.2)

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, truncating="post")
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/5
717/717 [==============================] - 125s 173ms/step - loss: 0.4163 - accuracy: 0.8069 - val_loss: 0.3772 - val_accuracy: 0.8301
Epoch 2/5
717/717 [==============================] - 127s 177ms/step - loss: 0.3193 - accuracy: 0.8638 - val_loss: 0.3786 - val_accuracy: 0.8397
Epoch 3/5
717/717 [==============================] - 127s 177ms/step - loss: 0.2763 - accuracy: 0.8855 - val_loss: 0.3878 - val_accuracy: 0.8397
Epoch 4/5
717/717 [==============================] - 127s 177ms/step - loss: 0.2432 - accuracy: 0.9030 - val_loss: 0.3641 - val_accuracy: 0.8399
Epoch 5/5
384/384 [==============================] - 19s 51ms/step - loss: 0.4386 - accuracy: 0.8347
Test Accuracy: 0.8346629738807678


In [14]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_length=100))
model.add(LSTM(200, return_sequences=True))  # Return sequences if stacking LSTM layers
model.add(Dropout(0.5))  # Add dropout layer with 30% dropout rate
model.add(LSTM(200))
model.add(Dropout(0.2))  # Add dropout layer with 20% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, validation_split=0.2)

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, truncating="post")
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/5
717/717 [==============================] - 126s 174ms/step - loss: 0.4231 - accuracy: 0.8046 - val_loss: 0.3729 - val_accuracy: 0.8390
Epoch 2/5
717/717 [==============================] - 132s 184ms/step - loss: 0.3225 - accuracy: 0.8610 - val_loss: 0.3648 - val_accuracy: 0.8400
Epoch 3/5
717/717 [==============================] - 132s 184ms/step - loss: 0.2865 - accuracy: 0.8813 - val_loss: 0.3601 - val_accuracy: 0.8387
Epoch 4/5
717/717 [==============================] - 130s 182ms/step - loss: 0.2585 - accuracy: 0.8945 - val_loss: 0.3984 - val_accuracy: 0.8400
Epoch 5/5
384/384 [==============================] - 19s 50ms/step - loss: 0.4120 - accuracy: 0.8372
Test Accuracy: 0.8371865749359131


In [18]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(LSTM(128, return_sequences=True))  # Return sequences if stacking LSTM layers
model.add(Dropout(0.5))  # Add dropout layer with 50% dropout rate
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))  # Add dropout layer with 50% dropout rate
model.add(LSTM(64))
model.add(Dropout(0.2))  # Add dropout layer with 20% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
717/717 [==============================] - 108s 148ms/step - loss: 0.4224 - accuracy: 0.8060 - val_loss: 0.3648 - val_accuracy: 0.8373
Epoch 2/10
717/717 [==============================] - 112s 156ms/step - loss: 0.3184 - accuracy: 0.8654 - val_loss: 0.3502 - val_accuracy: 0.8470
Epoch 3/10
717/717 [==============================] - 107s 149ms/step - loss: 0.2737 - accuracy: 0.8882 - val_loss: 0.3765 - val_accuracy: 0.8425
Epoch 4/10
717/717 [==============================] - 106s 147ms/step - loss: 0.2364 - accuracy: 0.9051 - val_loss: 0.3936 - val_accuracy: 0.8390
Epoch 5/10
717/717 [==============================] - 106s 147ms/step - loss: 0.2036 - accuracy: 0.9200 - val_loss: 0.4412 - val_accuracy: 0.8294
Epoch 6/10
717/717 [==============================] - 106s 148ms/step - loss: 0.1715 - accuracy: 0.9338 - val_loss: 0.5618 - val_accuracy: 0.8268
Epoch 7/10
717/717 [==============================] - 106s 148ms/step - loss: 0.1433 - accuracy: 0.9466 - val_loss: 0.5926 -

## Convolution Neural Network

In [20]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(64, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, validation_split=0.2)

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/5
717/717 [==============================] - 2s 2ms/step - loss: 0.4586 - accuracy: 0.7769 - val_loss: 0.3877 - val_accuracy: 0.8217
Epoch 2/5
717/717 [==============================] - 2s 2ms/step - loss: 0.3317 - accuracy: 0.8533 - val_loss: 0.3694 - val_accuracy: 0.8319
Epoch 3/5
717/717 [==============================] - 2s 2ms/step - loss: 0.2630 - accuracy: 0.8909 - val_loss: 0.3751 - val_accuracy: 0.8353
Epoch 4/5
717/717 [==============================] - 2s 2ms/step - loss: 0.2042 - accuracy: 0.9197 - val_loss: 0.3992 - val_accuracy: 0.8310
Epoch 5/5
384/384 [==============================] - 0s 665us/step - loss: 0.4438 - accuracy: 0.8258
Test Accuracy: 0.8257896304130554


In [32]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(64, 3, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
717/717 [==============================] - 2s 3ms/step - loss: 0.5565 - accuracy: 0.7535 - val_loss: 0.4997 - val_accuracy: 0.7940
Epoch 2/10
717/717 [==============================] - 2s 3ms/step - loss: 0.4196 - accuracy: 0.8221 - val_loss: 0.4756 - val_accuracy: 0.8107
Epoch 3/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3778 - accuracy: 0.8417 - val_loss: 0.4565 - val_accuracy: 0.8141
Epoch 4/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3525 - accuracy: 0.8534 - val_loss: 0.4491 - val_accuracy: 0.8128
Epoch 5/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3320 - accuracy: 0.8640 - val_loss: 0.4530 - val_accuracy: 0.8125
Epoch 6/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3158 - accuracy: 0.8747 - val_loss: 0.4474 - val_accuracy: 0.8113
Epoch 7/10
717/717 [==============================] - 2s 3ms/step - loss: 0.2997 - accuracy: 0.8826 - val_loss: 0.4469 - val_accuracy: 0.8100
Epoch 

In [35]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))  # Add dropout layer with 50% dropout rate
model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))  # Add dropout layer with 20% dropout rate
model.add(Dense(1, activation="sigmoid"))


# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train_padded, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
717/717 [==============================] - 3s 4ms/step - loss: 0.5556 - accuracy: 0.7736 - val_loss: 0.4184 - val_accuracy: 0.8193
Epoch 2/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3774 - accuracy: 0.8402 - val_loss: 0.3988 - val_accuracy: 0.8210
Epoch 3/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3187 - accuracy: 0.8724 - val_loss: 0.3933 - val_accuracy: 0.8285
Epoch 4/100
717/717 [==============================] - 3s 4ms/step - loss: 0.2666 - accuracy: 0.9005 - val_loss: 0.4213 - val_accuracy: 0.8202
Epoch 5/100
717/717 [==============================] - 3s 4ms/step - loss: 0.2163 - accuracy: 0.9243 - val_loss: 0.4918 - val_accuracy: 0.8128
Epoch 6/100
384/384 [==============================] - 0s 1ms/step - loss: 0.3989 - accuracy: 0.8271
Test Accuracy: 0.827092170715332


In [36]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))  # Add dropout layer with 50% dropout rate
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))  # Add dropout layer with 40% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train_padded, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
717/717 [==============================] - 3s 4ms/step - loss: 0.4649 - accuracy: 0.7800 - val_loss: 0.3800 - val_accuracy: 0.8263
Epoch 2/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3427 - accuracy: 0.8522 - val_loss: 0.3700 - val_accuracy: 0.8285
Epoch 3/100
717/717 [==============================] - 3s 4ms/step - loss: 0.2841 - accuracy: 0.8801 - val_loss: 0.3960 - val_accuracy: 0.8271
Epoch 4/100
717/717 [==============================] - 3s 4ms/step - loss: 0.2189 - accuracy: 0.9115 - val_loss: 0.4090 - val_accuracy: 0.8252
Epoch 5/100
384/384 [==============================] - 0s 971us/step - loss: 0.3729 - accuracy: 0.8280
Test Accuracy: 0.8279876112937927


In [37]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))  # Add dropout layer with 50% dropout rate
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.4))  # Add dropout layer with 40% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train_padded, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
717/717 [==============================] - 3s 4ms/step - loss: 0.5640 - accuracy: 0.7724 - val_loss: 0.4038 - val_accuracy: 0.8259
Epoch 2/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3845 - accuracy: 0.8398 - val_loss: 0.3874 - val_accuracy: 0.8332
Epoch 3/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3278 - accuracy: 0.8698 - val_loss: 0.4167 - val_accuracy: 0.8303
Epoch 4/100
717/717 [==============================] - 3s 4ms/step - loss: 0.2767 - accuracy: 0.8973 - val_loss: 0.4158 - val_accuracy: 0.8294
Epoch 5/100
384/384 [==============================] - 0s 959us/step - loss: 0.3958 - accuracy: 0.8298
Test Accuracy: 0.8297785520553589


In [39]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(128, 5, activation='relu', kernel_regularizer=l2(0.01)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))  # Add dropout layer with 30% dropout rate
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))  # Add dropout layer with 10% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train_padded, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
717/717 [==============================] - 3s 4ms/step - loss: 0.5132 - accuracy: 0.7770 - val_loss: 0.4539 - val_accuracy: 0.8095
Epoch 2/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3951 - accuracy: 0.8387 - val_loss: 0.4073 - val_accuracy: 0.8277
Epoch 3/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3485 - accuracy: 0.8630 - val_loss: 0.4086 - val_accuracy: 0.8287
Epoch 4/100
717/717 [==============================] - 3s 4ms/step - loss: 0.3056 - accuracy: 0.8869 - val_loss: 0.4372 - val_accuracy: 0.8212
Epoch 5/100
384/384 [==============================] - 0s 956us/step - loss: 0.4064 - accuracy: 0.8299
Test Accuracy: 0.8298599720001221


In [41]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))  # Add dropout layer with 30% dropout rate
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))  # Add dropout layer with 10% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train_padded, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
717/717 [==============================] - 2s 3ms/step - loss: 0.4590 - accuracy: 0.7818 - val_loss: 0.3727 - val_accuracy: 0.8313
Epoch 2/100
717/717 [==============================] - 2s 3ms/step - loss: 0.3361 - accuracy: 0.8535 - val_loss: 0.3754 - val_accuracy: 0.8343
Epoch 3/100
717/717 [==============================] - 2s 3ms/step - loss: 0.2628 - accuracy: 0.8938 - val_loss: 0.3818 - val_accuracy: 0.8332
Epoch 4/100
384/384 [==============================] - 0s 746us/step - loss: 0.3745 - accuracy: 0.8317
Test Accuracy: 0.8317323327064514


In [41]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))  # Add dropout layer with 30% dropout rate
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))  # Add dropout layer with 10% dropout rate
model.add(Dense(1, activation="sigmoid"))

# Compile the model

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train_padded, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Evaluate on the test set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
717/717 [==============================] - 2s 3ms/step - loss: 0.4590 - accuracy: 0.7818 - val_loss: 0.3727 - val_accuracy: 0.8313
Epoch 2/100
717/717 [==============================] - 2s 3ms/step - loss: 0.3361 - accuracy: 0.8535 - val_loss: 0.3754 - val_accuracy: 0.8343
Epoch 3/100
717/717 [==============================] - 2s 3ms/step - loss: 0.2628 - accuracy: 0.8938 - val_loss: 0.3818 - val_accuracy: 0.8332
Epoch 4/100
384/384 [==============================] - 0s 746us/step - loss: 0.3745 - accuracy: 0.8317
Test Accuracy: 0.8317323327064514


In [44]:
# Example: Remove Conv1D layer
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
717/717 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7284 - val_loss: 0.4412 - val_accuracy: 0.8071
Epoch 2/10
717/717 [==============================] - 1s 1ms/step - loss: 0.4672 - accuracy: 0.7818 - val_loss: 0.4006 - val_accuracy: 0.8238
Epoch 3/10
717/717 [==============================] - 1s 1ms/step - loss: 0.4355 - accuracy: 0.8001 - val_loss: 0.3905 - val_accuracy: 0.8339
Epoch 4/10
717/717 [==============================] - 1s 1ms/step - loss: 0.4066 - accuracy: 0.8160 - val_loss: 0.3888 - val_accuracy: 0.8303
Epoch 5/10
717/717 [==============================] - 1s 1ms/step - loss: 0.3851 - accuracy: 0.8255 - val_loss: 0.3780 - val_accuracy: 0.8299
Epoch 6/10
717/717 [==============================] - 1s 1ms/step - loss: 0.3594 - accuracy: 0.8414 - val_loss: 0.3751 - val_accuracy: 0.8284
Epoch 7/10
717/717 [==============================] - 1s 1ms/step - loss: 0.3426 - accuracy: 0.8503 - val_loss: 0.3753 - val_accuracy: 0.8294
Epoch 

In [49]:
# Example: Remove Conv1D layer
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=100))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.7))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
717/717 [==============================] - 2s 2ms/step - loss: 0.5681 - accuracy: 0.7131 - val_loss: 0.4649 - val_accuracy: 0.7907
Epoch 2/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4909 - accuracy: 0.7694 - val_loss: 0.4135 - val_accuracy: 0.8168
Epoch 3/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4614 - accuracy: 0.7857 - val_loss: 0.3969 - val_accuracy: 0.8200
Epoch 4/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4344 - accuracy: 0.8014 - val_loss: 0.3900 - val_accuracy: 0.8252
Epoch 5/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4164 - accuracy: 0.8111 - val_loss: 0.3821 - val_accuracy: 0.8303
Epoch 6/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4020 - accuracy: 0.8155 - val_loss: 0.3777 - val_accuracy: 0.8303
Epoch 7/10
717/717 [==============================] - 1s 2ms/step - loss: 0.3861 - accuracy: 0.8235 - val_loss: 0.3814 - val_accuracy: 0.8294
Epoch 

In [50]:
# Example: Remove Conv1D layer
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.7))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
717/717 [==============================] - 3s 3ms/step - loss: 0.5421 - accuracy: 0.7287 - val_loss: 0.4235 - val_accuracy: 0.8069
Epoch 2/10
717/717 [==============================] - 2s 3ms/step - loss: 0.4572 - accuracy: 0.7869 - val_loss: 0.3836 - val_accuracy: 0.8322
Epoch 3/10
717/717 [==============================] - 2s 3ms/step - loss: 0.4210 - accuracy: 0.8043 - val_loss: 0.3742 - val_accuracy: 0.8346
Epoch 4/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3946 - accuracy: 0.8230 - val_loss: 0.3661 - val_accuracy: 0.8360
Epoch 5/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3697 - accuracy: 0.8334 - val_loss: 0.3589 - val_accuracy: 0.8383
Epoch 6/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3510 - accuracy: 0.8446 - val_loss: 0.3570 - val_accuracy: 0.8383
Epoch 7/10
717/717 [==============================] - 2s 3ms/step - loss: 0.3347 - accuracy: 0.8535 - val_loss: 0.3548 - val_accuracy: 0.8383
Epoch 

In [56]:
# Example: Remove Conv1D layer
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=100))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.7))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
717/717 [==============================] - 2s 2ms/step - loss: 0.5725 - accuracy: 0.7080 - val_loss: 0.4745 - val_accuracy: 0.7769
Epoch 2/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4951 - accuracy: 0.7659 - val_loss: 0.4175 - val_accuracy: 0.8090
Epoch 3/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4667 - accuracy: 0.7846 - val_loss: 0.4042 - val_accuracy: 0.8224
Epoch 4/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4425 - accuracy: 0.7975 - val_loss: 0.3956 - val_accuracy: 0.8277
Epoch 5/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4195 - accuracy: 0.8089 - val_loss: 0.3841 - val_accuracy: 0.8301
Epoch 6/10
717/717 [==============================] - 1s 2ms/step - loss: 0.4048 - accuracy: 0.8146 - val_loss: 0.3853 - val_accuracy: 0.8315
Epoch 7/10
717/717 [==============================] - 1s 2ms/step - loss: 0.3879 - accuracy: 0.8266 - val_loss: 0.3795 - val_accuracy: 0.8336
Epoch 

In [57]:
# Example: Remove Conv1D layer
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=100))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.7))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train_padded, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded_3D = X_test_padded.reshape((X_test_padded.shape[0], X_test_padded.shape[1], 1))
accuracy = model.evaluate(X_test_padded, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/100
717/717 [==============================] - 2s 2ms/step - loss: 0.5691 - accuracy: 0.7107 - val_loss: 0.4777 - val_accuracy: 0.7717
Epoch 2/100
717/717 [==============================] - 2s 2ms/step - loss: 0.5002 - accuracy: 0.7655 - val_loss: 0.4212 - val_accuracy: 0.8134
Epoch 3/100
717/717 [==============================] - 2s 2ms/step - loss: 0.4660 - accuracy: 0.7841 - val_loss: 0.3995 - val_accuracy: 0.8235
Epoch 4/100
717/717 [==============================] - 1s 2ms/step - loss: 0.4428 - accuracy: 0.7981 - val_loss: 0.3889 - val_accuracy: 0.8275
Epoch 5/100
717/717 [==============================] - 1s 2ms/step - loss: 0.4265 - accuracy: 0.8075 - val_loss: 0.3930 - val_accuracy: 0.8298
Epoch 6/100
717/717 [==============================] - 1s 2ms/step - loss: 0.4006 - accuracy: 0.8191 - val_loss: 0.3849 - val_accuracy: 0.8298
Epoch 7/100
717/717 [==============================] - 1s 2ms/step - loss: 0.3868 - accuracy: 0.8278 - val_loss: 0.3805 - val_accuracy: 0.8296